In [19]:
#TODO: Switch time.sleep to selenium wait
#TODO: Switch xpath to css selector where possible for more robust code
#TODO: add condition to look for dates entered if input is given
#TODO: Use last (oldest) deed in archive to judge if deed can be added

#returns poorly ordered list of deeds from one date to another

def masslandrecords_deed_scraper(county_registry_url):

    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    import selenium
    import time
    wait_time = 1.75
    
    #Initialize list to store Deed Data
    street_address = ["Street_address"]
    town = ["Town"]
    parties = ["Parties"]
    document_num = ["document Number"]
    record_date = ["Recording Date"]
    record_time = ["Recording Time"]
    book_pages = ["Book Pages"]
    consideration = ["Consideration ($)"]
    document_status = ["Document Status"]
    
    #Open and Initialize Website
    driver = webdriver.Chrome()
    driver.get(county_registry_url)
    driver.implicitly_wait(10)
    time.sleep(wait_time)

    #Open Search Criteria Menu
    criteria = driver.find_element_by_id("Navigator1_SearchCriteria1_menuActiveSpace")
    criteria.click()

    #Select Recorded Land -> Recorded Date Search
    date_search = driver.find_element_by_id("Navigator1_SearchCriteria1_LinkButton04")
    date_search.click()

    #Enter Start Date
    start_date = driver.find_element_by_xpath("//input[@name='SearchFormEx1$ACSTextBox_DateFrom']")
    start_date.click()
    for i in range(2):
        start_date.send_keys(Keys.ARROW_RIGHT)
    for i in range(10):
        start_date.send_keys(Keys.BACKSPACE)
    start_date.send_keys("4/28/2020")

    #Enter End Date
    end_date = driver.find_element_by_xpath("//input[@name='SearchFormEx1$ACSTextBox_DateTo']")
    end_date.click()
    for i in range(2):
        end_date.send_keys(Keys.ARROW_RIGHT)
    for i in range(10):
        end_date.send_keys(Keys.BACKSPACE)
    end_date.send_keys("5/2/2020")

    #Select DEED as document type
    from selenium.webdriver.support.ui import Select
    select_deed = Select(driver.find_element_by_id('SearchFormEx1_ACSDropDownList_DocumentType'))
    select_deed.select_by_visible_text("DEED")

    #Initiate Deed Search
    search = driver.find_element_by_xpath("//input[@name='SearchFormEx1$btnSearch']")
    search.click()
    
    #Sorts Deed Entries by Most Recent, time inverted
    time.sleep(wait_time)
    sort_date = driver.find_element_by_xpath('//*[@id="DocList1_ContentContainer1"]/table/tbody/tr[1]/td/div/div[1]/table/thead/tr/th[2]/a[1]')
    sort_date.click()
    
    #Show 100 Deeds/Page
    try:
        show_100 = driver.find_element_by_id('DocList1_PageView100Btn')
        show_100.click()
    except:
        pass

    ################################### END PAGE INITIALIZATION ###################################
    
    # Loops through each page containing deed entries
    page_ct = 0
    total_page_list = driver.find_elements_by_xpath("/html/body/form/div[4]/div[27]/div[1]/div[2]/table/tbody/tr[1]/td/div/div[3]/table/tbody/tr/td[3]/a")
    for page in total_page_list:
        
        # Initializes raw deed list for the given page
        page_ct = page_ct + 1
        time.sleep(wait_time)
        #raw_deed_list = driver.find_elements_by_xpath("//*[@id='DocList1_ContentContainer1']/table/tbody/tr[1]/td/div/div[2]/table/tbody/tr")
        raw_deed_list = driver.find_elements_by_css_selector('#DocList1_ContentContainer1 > table > tbody > tr:nth-child(1) > td > div > div:nth-child(2) > table > tbody > tr')
        
        # Loops through each deed entry on page
        for entry in range(len(raw_deed_list)):         

            #Record town
            deed_town_str = "//*[@id='DocList1_ContentContainer1']/table/tbody/tr[1]/td/div/div[2]/table/tbody/tr[" + str(entry+1) + "]/td[5]"
            deed_town_info = driver.find_element_by_xpath(deed_town_str)
            town.append(deed_town_info.text)

            time.sleep(wait_time)

            #Opens deed entry
            stale = True
            while(stale):
                try:
                    open_deed_str = "//*[@id='DocList1_ContentContainer1']/table/tbody/tr[1]/td/div/div[2]/table/tbody/tr[" + str(entry+1) + "]/td[6]/a"
                    open_deed = driver.find_element_by_xpath(open_deed_str)
                    open_deed.click()
                    stale = False
                except:
                    print("Error: Open Deed Click Exception")
                    pass

            #TODO: Allows the page to register open the new deed window, may be better to use Selenium waits instead
            time.sleep(wait_time)

            #Record deed info from table
            deed_info = driver.find_elements_by_xpath("//*[@id='DocDetails1_GridView_Details']/tbody/tr[2]/td")      
            deed_info_comps = []
            stale = True
            while(stale):
                deed_info_comps = []
                for elm in deed_info:
                    try:
                        deed_info_comps.append(elm.text)
                        stale = False
                    except:
                        print("Error: Deed Info Failure")
                        pass
            document_num.append(deed_info_comps[0])
            record_date.append(deed_info_comps[1])
            record_time.append(deed_info_comps[2])
            book_pages.append(deed_info_comps[5])
            consideration.append(deed_info_comps[6])
            document_status.append(deed_info_comps[7])

            #Records deed address from table
            deed_multi_address = driver.find_elements_by_xpath("//*[@id='DocDetails1_GridView_Property']/tbody/tr")
            deed_address_temp = []
            address_present = True
            for i in range(len(deed_multi_address)):
                deed_address_path = "//*[@id='DocDetails1_GridView_Property']/tbody/tr[" + str(i+1) + "]"
                deed_address = driver.find_elements_by_xpath(deed_address_path)

                for entry in deed_address:
                    address_temp = entry.text
                
                try:
                    deed_address_temp.append(address_temp)
                except: #Sometimes address isn't listed
                    print("Error: Deed address not detected")
                    address_present = False
            
            if(address_present):
                try:
                    del deed_address_temp[0]
                except:
                    print("Error: First Address Not Present")
                    pass
                street_address_comps = []
                
            if (len(deed_address_temp) > 0):
                for i in range(len(deed_address_temp)):
                    street_address_comps.append(deed_address_temp[i])
                street_address.append(street_address_comps)
            else:
                street_address.append('N/A')

            #Prints Grantors (sellers) and Grantees (Buyers)
            deed_parties = driver.find_elements_by_xpath("//*[@id='DocDetails1_GridView_GrantorGrantee']/tbody/tr")
            parties_temp = []
            emptySet = 1
            for elm in deed_parties:
                if (emptySet == 1):
                    emptySet = 0
                else:
                    stale = True
                    while(stale):
                        try:
                            parties_temp.append(elm.text)
                            stale = False
                        except:
                            print("Error: Append Parties Failure")
                            pass
            parties.append(parties_temp)
            
        #Go to next page
        if (page_ct < len(total_page_list)):
            stale = True
            while(stale):
                select_next_page = driver.find_element_by_id('DocList1_LinkButtonNext')
                try:
                    select_next_page.click()
                    stale = False
                except:
                    print("Error: Next Page Failure")
                    pass

    deed_list = [street_address,town,parties,consideration,document_status,record_date,record_time,document_num,book_pages]
    driver.close()
    return deed_list

In [21]:
#registry = 'http://www.masslandrecords.com/MiddlesexSouth/'
registry = 'http://www.masslandrecords.com/Nantucket/'

deed_list_MiddlesexSouth = masslandrecords_deed_scraper(registry)


Error: Deed Info Failure
Error: Deed Info Failure
Error: Deed Info Failure
Error: Deed Info Failure
Error: Deed Info Failure
Error: Deed Info Failure
Error: Deed Info Failure


IndexError: list index out of range

In [4]:
print(deed_list_MiddlesexSouth)

[['Street_address', ['72 WOLCOTT ST  '], ['10 POOLE ST  '], ['117-119 NORWOOD AVE UNIT 117'], ['750 COMMONWEALTH AVE  '], ['51 COLONIAL RD  '], ['26 TEMPLE PLC  '], ['24 SARGENT ST UNIT 24'], ['24 SARGENT ST UNIT 24'], ['1 CRANBERRY LN BLDG 4 UNIT 36'], ['26 SARGENT RD  '], ['74 DEER PATH  '], ['25 CARVER ST UNIT 1'], ['23 MONMOUTH ST  '], ['36 SHADOW CREEK LN  '], ['191 WILLOWGATE RISE  '], ['388-390 WASHINGTON ST UNIT 2'], ['24 GIRARD ST  '], ['24 GIRARD ST  '], ['66 SALEM END LN  '], ['    LOTS 3625, 3626-3627 AND 3629'], ['120 SOUTH MAIN ST  '], ['22 BLOSSOMCREST RD  '], ['5 LIBERTY HILL CIR  '], ['155 NEWTOWN RD  '], ['2008 SYMMES CIR UNIT 2008'], ['44 SOLON ST  '], ['118 PARKER ST UNIT C-13'], ['3 SUMMIT DR UNIT 308'], ['97 BOW ST  '], ['113 PLEASANT ST  '], ['11 WOODPARK CIR  '], ['245 PRENTICE ST  '], ['319 ASHLAND ST  '], ['34 GATEWOOD STREET  '], ['84 LINDEN AVE  '], ['38 YORK AVE  '], ['9 HAMMOND PL UNIT 9 GARAGE SP S1'], ['20 BLUE JAY LN  '], ['7 DRUMMER BOY WAY UNIT 7'], [

In [8]:
def county_deed_arrange(raw_deed_list):
    deed_master_list = []
    for j in range(len(raw_deed_list[0])):
        deed_entries = []
        for k in range(len(raw_deed_list)):
            deed_entries.append(raw_deed_list[k][j])
        deed_master_list.append(deed_entries)
    return deed_master_list

In [9]:
def reverse_deed_list_times(archive_list, deed_master_list):

    new_day = []
    sorting_dates = []

    for deed in range(len(deed_master_list)-1):
        date_list_unplaced = deed_master_list[deed+1][5].split("/")
        appended = False

        for date in range(len(sorting_dates)):
            date_list_placed = sorting_dates[date][0][5].split("/")

            if date_list_unplaced[2] == date_list_placed[2]:
                if date_list_unplaced[0] == date_list_placed[0]:
                    if date_list_unplaced[1] == date_list_placed[1]:
                        sorting_dates[date].append(deed_master_list[deed+1])
                        appended = True

        if (appended == False):
            new_day = []
            new_day.append(deed_master_list[deed+1])
            sorting_dates.append(new_day)

    for i in range(len(sorting_dates)):
        for j in range(len(sorting_dates[len(sorting_dates)-i-1])):
            archive_list.insert(0, sorting_dates[len(sorting_dates)-i-1][j])
    
    return archive_list

In [10]:
old_list = []
additions_to_list = county_deed_arrange(deed_list_MiddlesexSouth)
updated_list = reverse_deed_list_times(old_list, additions_to_list)

for i in range(len(updated_list)):
    print(updated_list[i])
    print("")

[['621 WATERTOWN ST UNIT 621-18'], 'NEWTON', ['ZACEK JUDITH C Grantor', 'ARCHER JESSICA L AKER Grantee', 'ARCHER MATTHEW Grantee'], '525000.00', 'In workflow', '05/01/2020', '3:31PM', '66170', '74595/481']

[['33 MOORE RD  '], 'SOMERVILLE', ['STANKSY KATHLEEN J Grantor', 'HARRIS MARIE SUZAN Grantee', 'HILLAIRE GARRON Grantee'], '745000.00', 'In workflow', '05/01/2020', '3:25PM', '66148', '74595/375']

[['33 MOORE RD  '], 'WAYLAND', ['STANKSY KATHLEEN J Grantor', 'HARRIS MARIE SUZAN Grantee', 'HILLAIRE GARRON Grantee'], '745000.00', 'In workflow', '05/01/2020', '3:25PM', '66148', '74595/375']

[['28 CENTER VILLAGE DR UNIT 28'], 'CONCORD', ['ANDREWS JOSEPH L Grantor', 'CASEY KATHLEEN MARIE Grantee'], '569000.00', 'In workflow', '05/01/2020', '3:19PM', '66133', '74595/287']

[['38-40 WALKER ST UNIT 38'], 'SOMERVILLE', ['MYCYNEK STEVEN V Grantor', 'MYCNEK RIMA C Grantor', 'KOSSAK ALEXANDER E Grantee', 'KOSSAK CHRIS M Grantee'], '650000.00', 'In workflow', '05/01/2020', '3:18PM', '66120', '

In [11]:
import pickle
county_name = 'MiddlesexSouth'
archive_path = 'archives/' + county_name + '_archive_list' ###DEBUG so I dont delete whole list
with open(archive_path, 'wb') as f:
     pickle.dump(updated_list, f)

In [16]:
import pickle
county_name = 'Nantucket'
archive_path = 'archives/' + county_name + '_archive_list'
with open(archive_path, 'rb') as f:
     mylist = pickle.load(f)
for i in range(len(mylist)):
    print(mylist[i]) 
    print("")

[['3 EAST TRISTRAM AVENUE LOT 3 PLFL 7-F', '3R EAST TRISTRAM AVENUE  '], 'NANTUCKET', ['GUND GORDON TR Grantor', '3 EAST TRISTRAM TRUST Grantor', 'GREAT MEADOWS LLC Grantee'], '0.00', 'Verified/Certified', '05/06/2020', '11:26AM', '1060', '01744/288']

[['73 BAXTER ROAD LOTS 34 & 35 PLBK 2 PG 81'], 'NANTUCKET', ['DARBY CHRISTIAN M Grantor', 'THOMPSON BRUCE Grantee', 'THOMPSON MARY Grantee'], '1807000.00', 'Verified/Certified', '05/01/2020', '11:34AM', '1035', '01744/135']

[['1 PERRY LANE LOT 3 PLAN 2007-31', '26 BARTLETT ROAD LOT 2 PLAN 2007-31'], 'NANTUCKET', ['NEWMAN JOHN L Grantor', 'NEWMAN JOHN L TR Grantee', 'NEWMAN FAMILY TRUST - 2020 Grantee'], '0.00', 'Verified/Certified', '04/30/2020', '11:57AM', '1021', '01744/98']

[['3 HOOPER FARM ROAD LOT B-3 PLFL 26-E'], 'NANTUCKET', ['HOOPER FARM LLC Grantor', 'WILSON KRISTIE Grantee', 'WILSON MICHAEL J Grantee'], '1.00', 'Verified/Certified', '04/28/2020', '11:32AM', '1008', '01743/317']

[['17 NEW LANE LOT 2 PLFL 45-J'], 'NANTUCKET', 